In [27]:
import requests
import passwords as pw
import pandas as pd
import time
from sklearn.preprocessing import MultiLabelBinarizer


In [28]:
def get_puuid(game_name, tag_line, api_key):
    api_url = (f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key=" + api_key)
    resp = requests.get(api_url)
    player_info = resp.json()
    return player_info['puuid']


In [29]:
get_puuid('I will trade', 'NA1', pw.API_KEY)

'I1b23fOoa0-kFyLWIo00nUYJPn9eJGaDMWcZ6fZkazuslrcJtunBf_UtXyMnPEfpWpOy2W5n1R56sw'

In [30]:
def get_active_game(puuid, api_key):
    headers = {"X-Riot-Token": api_key}
    api_url = (f"https://na1.api.riotgames.com/lol/spectator/v5/active-games/by-summoner/{puuid}")
    resp = requests.get(api_url, headers = headers)
    game_data = resp.json()

    champion_ids = [player['championId'] for player in game_data['participants']]
    return champion_ids


In [31]:
def champion_names(champ_ids):
    version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
    api_url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json"
    champs = requests.get(api_url).json()["data"]
    id_to_name = {int(name["key"]): champ for champ, name in champs.items()}
    
    champ_names = [id_to_name.get(id, "Unknown") for id in champ_ids]
    print("Champions in game:", champ_names)

In [32]:
def get_match_ids(start, count, puuid, api_key):
    api_url = (f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}&api_key={api_key}")
    resp = requests.get(api_url)
    match_ids = resp.json()
    return match_ids

In [33]:
def get_match_data(match_id, api_key):
    api_url = ("https://americas.api.riotgames.com/lol/match/v5/matches/" + match_id +
               "?api_key=" + api_key)
    try:
        resp = requests.get(api_url)
        match_data = resp.json()
        return match_data
    except:
        return None

In [34]:
def pull_matches(start, count = 100):
    dataframe = []
    match_ids = get_match_ids(start, count, pw.spica_PUUID, pw.API_KEY)
    print(match_ids)
    for match in match_ids:
        match_data = get_match_data(match, pw.API_KEY)
        if not match_data or 'info' not in match_data:
            continue

        participants = match_data['info']['participants']
        
        player = next((p for p in participants if p['puuid'] == pw.spica_PUUID), None)
        player_team = player['teamId']
        player_win = player['win']
        
        ally_champs = [p['championName'] for p in participants if p['teamId'] == player_team]
        enemy_champs = [p['championName'] for p in participants if p['teamId'] != player_team]
        
        dataframe.append({
            'match_id': match,
            'ally_champs': ally_champs,
            'enemy_champs': enemy_champs,
            'win': int(player_win)
        })
    
    return pd.DataFrame(dataframe)






In [35]:
final_df = pd.DataFrame(columns = ['match_id', 'ally_champs', 'enemy_champs'])

In [36]:
iterations = 10
start = 0
for i in range(iterations):
    temp = pull_matches(start, 100)
    start += 100
    final_df = pd.concat([temp, final_df], ignore_index = True)
    time.sleep(120)


['NA1_5318448730', 'NA1_5318071133', 'NA1_5318069738', 'NA1_5318062264', 'NA1_5317851480', 'NA1_5317476803', 'NA1_5317467194', 'NA1_5317457917', 'NA1_5316863136', 'NA1_5316855346', 'NA1_5316846021', 'NA1_5316814130', 'NA1_5316804270', 'NA1_5316788086', 'NA1_5316766499', 'NA1_5316708545', 'NA1_5316666871', 'NA1_5316232033', 'NA1_5316224830', 'NA1_5316049564', 'NA1_5316040463', 'NA1_5316027233', 'NA1_5316006618', 'NA1_5315992988', 'NA1_5315976545', 'NA1_5315626155', 'NA1_5315622825', 'NA1_5315614814', 'NA1_5315595784', 'NA1_5315569476', 'NA1_5315540969', 'NA1_5315509869', 'NA1_5315462872', 'NA1_5315412117', 'NA1_5314988488', 'NA1_5314953539', 'NA1_5314935661', 'NA1_5314905819', 'NA1_5314858937', 'NA1_5314821691', 'NA1_5314792726', 'NA1_5314762473', 'NA1_5314733004', 'NA1_5314696516', 'NA1_5314682264', 'NA1_5314424447', 'NA1_5314416775', 'NA1_5314400100', 'NA1_5314370641', 'NA1_5314345179', 'NA1_5314305530', 'NA1_5314267877', 'NA1_5314210478', 'NA1_5314183803', 'NA1_5314146239', 'NA1_5314

In [37]:
final_df

,match_id,ally_champs,enemy_champs,win
0,NA1_5185567216,"[Gnar, Sylas, Yone, Swain, Pyke]","[Irelia, Morgana, AurelionSol, Ezreal, Karma]",0.0
1,NA1_5180652078,"[Tryndamere, Sylas, Gragas, Jinx, Lulu]","[Olaf, Graves, Talon, Ekko, Zoe]",1.0
2,NA1_5180485659,"[Brand, Vi, Katarina, Ashe, Janna]","[Gwen, XinZhao, Yone, Jhin, Karma]",1.0
3,NA1_5179443546,"[Rengar, Warwick, Aurora, Kalista, Braum]","[Nocturne, Jayce, AurelionSol, Jinx, TahmKench]",0.0
4,NA1_5176501395,"[DrMundo, Sylas, Chogath, Thresh, Caitlyn]","[Shen, Viego, Naafiri, KogMaw, Janna]",0.0
...,...,...,...,...
804,NA1_5311690546,"[Teemo, LeeSin, Ahri, Sivir, Lulu]","[KSante, Pantheon, Sylas, Xayah, Elise]",1.0
805,NA1_5311380556,"[Kayle, Milio, Amumu, Xayah, Hwei]","[Ashe, Pantheon, Sejuani, Mordekaiser, Vladimir]",0.0
806,NA1_5311375664,"[Ziggs, Twitch, Kindred, Galio, Maokai]","[Syndra, Caitlyn, XinZhao, Senna, Ezreal]",0.0
807,NA1_5311369349,"[Tristana, Blitzcrank, Smolder, Malzahar, Monk...","[Vladimir, Ornn, DrMundo, Lux, Akshan]",1.0


In [38]:
def get_all_champs():
    version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
    api_url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json"

    champs_url = f'https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json'
    resp = requests.get(champs_url).json()

    champ_data = resp['data']
    champ_names = list(champ_data.keys())

    return champ_names

In [39]:
final_df = final_df.drop_duplicates(subset = ['match_id'])
final_df

,match_id,ally_champs,enemy_champs,win
0,NA1_5185567216,"[Gnar, Sylas, Yone, Swain, Pyke]","[Irelia, Morgana, AurelionSol, Ezreal, Karma]",0.0
1,NA1_5180652078,"[Tryndamere, Sylas, Gragas, Jinx, Lulu]","[Olaf, Graves, Talon, Ekko, Zoe]",1.0
2,NA1_5180485659,"[Brand, Vi, Katarina, Ashe, Janna]","[Gwen, XinZhao, Yone, Jhin, Karma]",1.0
3,NA1_5179443546,"[Rengar, Warwick, Aurora, Kalista, Braum]","[Nocturne, Jayce, AurelionSol, Jinx, TahmKench]",0.0
4,NA1_5176501395,"[DrMundo, Sylas, Chogath, Thresh, Caitlyn]","[Shen, Viego, Naafiri, KogMaw, Janna]",0.0
...,...,...,...,...
804,NA1_5311690546,"[Teemo, LeeSin, Ahri, Sivir, Lulu]","[KSante, Pantheon, Sylas, Xayah, Elise]",1.0
805,NA1_5311380556,"[Kayle, Milio, Amumu, Xayah, Hwei]","[Ashe, Pantheon, Sejuani, Mordekaiser, Vladimir]",0.0
806,NA1_5311375664,"[Ziggs, Twitch, Kindred, Galio, Maokai]","[Syndra, Caitlyn, XinZhao, Senna, Ezreal]",0.0
807,NA1_5311369349,"[Tristana, Blitzcrank, Smolder, Malzahar, Monk...","[Vladimir, Ornn, DrMundo, Lux, Akshan]",1.0


In [40]:
final_df.to_csv('matches_bwipo.csv', index = False)

In [41]:
#One Hot Encoding

all_champs = get_all_champs()

mlb_allies = MultiLabelBinarizer(classes=all_champs)
mlb_enemies = MultiLabelBinarizer(classes=all_champs)

ally_encoded = pd.DataFrame(mlb_allies.fit_transform(final_df['ally_champs']),
                            columns=[f'ally_{c}' for c in mlb_allies.classes_],
                            index=final_df.index)

enemy_encoded = pd.DataFrame(mlb_enemies.fit_transform(final_df['enemy_champs']),
                             columns=[f'enemy_{c}' for c in mlb_enemies.classes_],
                             index=final_df.index)

one_hot = pd.concat([final_df[['match_id']], ally_encoded, enemy_encoded, final_df['win']], axis=1)

one_hot


c:\Users\minaz\OneDrive\Desktop\personal\win_predictor\env\Lib\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['FiddleSticks'] will be ignored
  warnings.warn(


,match_id,ally_Aatrox,ally_Ahri,ally_Akali,ally_Akshan,ally_Alistar,ally_Ambessa,ally_Amumu,ally_Anivia,ally_Annie,...,enemy_Yorick,enemy_Yuumi,enemy_Zac,enemy_Zed,enemy_Zeri,enemy_Ziggs,enemy_Zilean,enemy_Zoe,enemy_Zyra,win
0,NA1_5185567216,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,NA1_5180652078,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1.0
2,NA1_5180485659,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
3,NA1_5179443546,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,NA1_5176501395,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,NA1_5311690546,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
805,NA1_5311380556,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.0
806,NA1_5311375664,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
807,NA1_5311369349,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [42]:
one_hot.to_csv('one_hot_bwipo.csv', index = False)

In [47]:
csv_files = ['matches_pob.csv', 'matches_dl.csv', 'matches_spica.csv', 'matches_stunt.csv', 'matches_bwipo.csv']

df_list = [pd.read_csv(file) for file in csv_files]

combined_df = pd.concat(df_list, ignore_index=True)

combined_df.to_csv('all_matches.csv', index = False)

In [48]:
csv_files = ['one_hot_pob.csv', 'one_hot_dl.csv', 'one_hot_spica.csv', 'one_hot_stunt.csv', 'one_hot_bwipo.csv']

df_list = [pd.read_csv(file) for file in csv_files]

combined_df = pd.concat(df_list, ignore_index=True)

combined_df.to_csv('all_one_hot.csv', index = False)